In [9]:
import requests
import json
import pandas as pd

from plaid import Client

In [2]:
# API_ROOT = "https://sandbox.plaid.com"
TOKEN_URL = "https://sandbox.plaid.com/sandbox/public_token/create"
PLAID_CLIENT_ID = "5ecc3b1e83a47600129f6d43"
PLAID_SECRET = "7f4ece2a8c3ccebd147d92a548bcaf"
PLAID_PUBLIC_KEY = "d92b2cb6724519c30c71daf2ac7358"

In [3]:
client = Client(
    client_id=PLAID_CLIENT_ID, 
    secret=PLAID_SECRET, 
    public_key=PLAID_PUBLIC_KEY, 
    environment='sandbox'
)

In [4]:
token_data = {
  "public_key": PLAID_PUBLIC_KEY,
  "institution_id": "ins_1",
  "initial_products": ["auth"],
  "options": {"webhook": "https://www.genericwebhookurl.com/webhook"},
}

response_tok = requests.post(TOKEN_URL, json = token_data, headers={
    "Content-Type": "application/json"
})

public_token = response_tok.json()["public_token"]

In [5]:
exchange_response = client.Item.public_token.exchange(public_token)
access_token = exchange_response['access_token']

In [7]:
transactions_response =  client.Transactions.get(access_token, '2019-12-01', '2020-05-27')

In [10]:
transactions_response = pd.DataFrame(transactions_response['transactions'])

In [12]:
categories = transactions_response['category'].apply(pd.Series)\
                                            .rename(columns = lambda x : 'cat_' + str(x))\
                                            .drop(['cat_2'],axis = 1)

In [13]:
transactions_final = pd.concat([transactions_response[:], categories[:]], axis=1)

In [14]:
transactions_final['amount'] = pd.to_numeric(transactions_final['amount'])
transactions_final['date'] = pd.to_datetime(transactions_final.date).dt.strftime('%m/%Y')

In [15]:
income = transactions_final[transactions_final['amount']>0]
expense = transactions_final[transactions_final['amount']<0]

In [16]:
income['cat_'] = 'Income'
expense['cat_'] = 'Expense'

C:\Users\t.linnik\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\t.linnik\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [17]:
income_final = income[['amount','category_id','date','cat_0','cat_1','cat_']].groupby(['category_id','date'])\
                                                                        .agg({'cat_':'first','cat_0':'first','cat_1':'first','amount':'sum'})\
                                                                        .reset_index()
expense_final = expense[['amount','category_id','date','cat_0','cat_1','cat_']].groupby(['category_id','date'])\
                                                                        .agg({'cat_':'first','cat_0':'first','cat_1':'first','amount':'sum'})\
                                                                        .reset_index()

In [18]:
cashflow = pd.merge(income_final,expense_final,how='outer')

In [19]:
cashflow_results = pd.pivot_table(cashflow, values = ['amount'], index=['cat_','cat_0','cat_1'], columns=['date'], aggfunc='sum')

In [20]:
cashflow_results['total']=cashflow_results.sum(axis=1)

In [21]:
cashflow_results

amount           \
date                                                   01/2020  02/2020   
cat_    cat_0          cat_1                                              
Expense Transfer       Credit                            -4.22    -4.22   
        Travel         Airlines and Aviation Services  -500.00  -500.00   
Income  Food and Drink Restaurants                     1105.73  1105.73   
        Payment        Credit Card                       25.00    25.00   
        Recreation     Gyms and Fitness Centers          78.50    78.50   
        Shops          Sporting Goods                   500.00   500.00   
        Transfer       Debit                           5850.00  5850.00   
                       Deposit                         1000.00  1000.00   
        Travel         Airlines and Aviation Services  1000.00      NaN   
                       Taxi                              11.73    11.73   

                                                                         \
date                                                   03/2020  04/2020   
cat_    cat_0          cat_1                                              
Expense Transfer       Credit                            -4.22    -4.22   
        Travel         Airlines and Aviation Services  -500.00  -500.00   
Income  Food and Drink Restaurants                     1105.73  1105.73   
        Payment        Credit Card                       25.00    25.00   
        Recreation     Gyms and Fitness Centers          78.50    78.50   
        Shops          Sporting Goods                   500.00   500.00   
        Transfer       Debit                           5850.00  5850.00   
                       Deposit                         1000.00  1000.00   
        Travel         Airlines and Aviation Services  1000.00   500.00   
                       Taxi                              11.73    11.73   

                                                                         \
date                                                   05/2020  12/2019   
cat_    cat_0          cat_1                                              
Expense Transfer       Credit                            -4.22    -4.22   
        Travel         Airlines and Aviation Services  -500.00  -500.00   
Income  Food and Drink Restaurants                     1105.73  1105.73   
        Payment        Credit Card                       25.00    25.00   
        Recreation     Gyms and Fitness Centers          78.50    78.50   
        Shops          Sporting Goods                   500.00   500.00   
        Transfer       Debit                           5850.00  5850.00   
                       Deposit                         1000.00  1000.00   
        Travel         Airlines and Aviation Services      NaN   500.00   
                       Taxi                               5.40    11.73   

                                                          total  
date                                                             
cat_    cat_0          cat_1                                     
Expense Transfer       Credit                            -25.32  
        Travel         Airlines and Aviation Services  -3000.00  
Income  Food and Drink Restaurants                      6634.38  
        Payment        Credit Card                       150.00  
        Recreation     Gyms and Fitness Centers          471.00  
        Shops          Sporting Goods                   3000.00  
        Transfer       Debit                           35100.00  
                       Deposit                          6000.00  
        Travel         Airlines and Aviation Services   3000.00  
                       Taxi                               64.05

In [22]:
cashflow_results.to_excel('plaid.xlsx')

In [23]:
transactions_raw_data =  client.Transactions.get(access_token, '2019-12-01', '2020-05-27')

In [35]:
df = pd.DataFrame(transactions_raw_data['transactions'])
df.to_csv("./file.csv", sep=',',index=False)